<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#SparkSession-생성" data-toc-modified-id="SparkSession-생성-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>SparkSession 생성</a></span></li><li><span><a href="#List에서-RDD-생성하기" data-toc-modified-id="List에서-RDD-생성하기-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>List에서 RDD 생성하기</a></span></li><li><span><a href="#파일에서-RDD-생성하기" data-toc-modified-id="파일에서-RDD-생성하기-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>파일에서 RDD 생성하기</a></span><ul class="toc-item"><li><span><a href="#RDD와-Spark-Dataframe을-만드는-함수는-다르다" data-toc-modified-id="RDD와-Spark-Dataframe을-만드는-함수는-다르다-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>RDD와 Spark Dataframe을 만드는 함수는 다르다</a></span></li></ul></li><li><span><a href="#CSV에서-RDD-생성" data-toc-modified-id="CSV에서-RDD-생성-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>CSV에서 RDD 생성</a></span></li></ul></div>

## SparkSession 생성
클라이언트 실행
* local = spark를 로컬에서 실행
* appName = 앱이름

In [1]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [2]:
print ("Spark version \t: {}".format(spark.version))
print ("Spark App \t: {}".format(spark.conf.get('spark.app.name')))
print ("Spark Master \t: {}".format(spark.conf.get('spark.master')))
print ("Spark Host \t: {}".format(spark.conf.get('spark.driver.host')))

Spark version 	: 3.0.3
Spark App 	: myApp
Spark Master 	: local
Spark Host 	: DESKTOP-7TC72N3


- user working directory

In [4]:
spark.conf.get('spark.sql.warehouse.dir')

'file:/C:/Users/Nunu/BigData/spark-warehouse'

- 모든 설정

In [5]:
spark.sparkContext.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.host', 'DESKTOP-7TC72N3'),
 ('spark.app.name', 'myApp'),
 ('spark.app.id', 'local-1634282701319'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.startTime', '1634282697259'),
 ('spark.driver.port', '59856'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [6]:
spark.sparkContext._conf.getAll()

[('spark.master', 'local'),
 ('spark.driver.host', 'DESKTOP-7TC72N3'),
 ('spark.app.name', 'myApp'),
 ('spark.app.id', 'local-1634282701319'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.startTime', '1634282697259'),
 ('spark.driver.port', '59856'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

- 설정 변경
SparkSession 을 만들고 난 후에 SparkContext를 경유해 `spark.sparkContext._conf.set()` 함수로 설정 변경 가능 

In [7]:
spark.sparkContext._conf.set("spark.driver.bindAddress", "127.0.0.1")

In [8]:
spark.sparkContext._conf.set("spark.executor.memory", '8g')
spark.sparkContext._conf.set('spark.app.name', 'yanu')
spark.sparkContext._conf.set('spark.executor.cores', '3')
spark.sparkContext._conf.set('spark.cores.max', '3')
spark.sparkContext._conf.set("spark.driver.memory",'8g')

In [9]:
spark.sparkContext._conf.getAll()

[('spark.master', 'local'),
 ('spark.app.name', 'yanu'),
 ('spark.driver.host', 'DESKTOP-7TC72N3'),
 ('spark.app.id', 'local-1634282701319'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '59856'),
 ('spark.cores.max', '3'),
 ('spark.driver.bindAddress', '127.0.0.1'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '8g'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.executor.memory', '8g'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.startTime', '1634282697259'),
 ('spark.executor.cores', '3'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

## List에서 RDD 생성하기
`sparkContext.paralleslize()`함수를 이용하여 List에서 RDD를 생성할 수 있다.

In [12]:
import pyspark
from pyspark import SparkConf, SparkContext
myList=[1,2,3,4,5,6,7]
myRdd1 = spark.sparkContext.parallelize(myList)

myRdd1.take(3)

[1, 2, 3]

* `parallelize([0, 2, 3, 4, 6], 2)` : slice의 개수를 정할 수 있다.
* `glom()` : partition에 있는 요소를 묶어 RDD를 만들어 준다  

In [14]:
spark.sparkContext.parallelize([0, 2, 3, 4, 6], 2).collect()

[0, 2, 3, 4, 6]

2개로 분할

In [15]:
spark.sparkContext.parallelize([1,2,3,4,5,6,7], 2).glom().collect()

[[1, 2, 3], [4, 5, 6, 7]]

## 파일에서 RDD 생성하기
wikipedia에서 Apache spark를 검색한 문단 일부분

In [16]:
%%writefile data/ds_spark_wiki.txt
Wikipedia
Apache Spark is an open source cluster computing framework.
아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
Apache Spark Apache Spark Apache Spark Apache Spark
아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크
Originally developed at the University of California, Berkeley's AMPLab,
the Spark codebase was later donated to the Apache Software Foundation,
which has maintained it since.
Spark provides an interface for programming entire clusters with
implicit data parallelism and fault-tolerance.

Overwriting data/ds_spark_wiki.txt


* List와 같이 SparkContext를 사용
* `textFile()` 함수 인자로 파일명을 넣는다

In [17]:
import os
myRdd2=spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_wiki.txt"))

* `first()` == `take(1)`

first()는 내부적으로 take(1)를 사용

In [16]:
myRdd2.first()

'Wikipedia'

### RDD와 Spark Dataframe을 만드는 함수는 다르다
* `SparkSession.sparkContext.textFile()` : 'SparkContext'를 사용하므로 RDD를 생성
* `SparkSession.read.text()` : DataFrame을 생성

In [18]:
import os
myDf=spark.read.text(os.path.join("data", "ds_spark_wiki.txt"))
print (myDf.first())

Row(value='Wikipedia')


In [19]:
print (type(myDf))

<class 'pyspark.sql.dataframe.DataFrame'>


## CSV에서 RDD 생성
csv 파일은 컴마로 구분된 데이터를 저장하고 있다

In [20]:
%%writefile ./data/ds_spark_2cols.csv
35, 2
40, 27
12, 38
15, 31
21, 1
14, 19
46, 1
10, 34
28, 3
48, 1
16, 2
30, 3
32, 2
48, 1
31, 2
22, 1
12, 3
39, 29
19, 37
25, 2

Overwriting ./data/ds_spark_2cols.csv


In [21]:
myRdd4 = spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_2cols.csv"))

- `take()`를 하면 결과는 리스트가 된다

In [22]:
myList=myRdd4.take(5)
print (type(myList))

<class 'list'>
